In [70]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from eden.util import configure_logging
import logging
logger = logging.getLogger()
configure_logging(logger,verbosity=0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
import cPickle as pickle
def save_data(data, fname='data.dat'):
    with open(fname, 'wb') as outfile:
        pickle.dump(data, outfile, pickle.HIGHEST_PROTOCOL)
    if len(data.shape)==1:
        print 'Saved %s (%d)' % (fname, data.shape[0])
    else:
        print 'Saved %s (%d,%d)' % (fname, data.shape[0], data.shape[1])

def load_data(fname='data.dat'):
    with open(fname, 'rb') as infile:
        data = pickle.load(infile)
    if len(data.shape)==1:
        print 'Loaded %s (%d)' % (fname, data.shape[0])
    else:
        print 'Loaded %s (%d,%d)' % (fname, data.shape[0], data.shape[1])
    return data

#Predicitve model performance evaluation

In [72]:
def train_perform(X_train, y_train, X_test,y_test):
    # Induce a predictive model
    print 'Training on data matrix [%d x %d]' %(X_train.shape)
    from sklearn.linear_model import SGDClassifier
    estimator = SGDClassifier(average=True, class_weight='auto', shuffle=True, n_jobs=-1)
    estimator.fit(X_train,y_train)

    # Print predictive performance
    from sklearn.metrics import confusion_matrix
    print 'Confusion matrix:'
    print(confusion_matrix(y_test, estimator.predict(X_test)))
    print
    from sklearn.metrics import classification_report
    print 'Classification Report:'
    print classification_report(y_test, estimator.predict(X_test))

#Data preparation

Split data into train/test according to binary clustering

In [73]:
def train_test_ids_split(X, confidence_threshold=1):
    from sklearn.cluster import MiniBatchKMeans
    kmeans = MiniBatchKMeans(n_clusters=2)
    classes = kmeans.fit_predict(X)
    
    from sklearn.linear_model import SGDClassifier
    estimator = SGDClassifier(average=True, class_weight='auto', shuffle=True, n_jobs=-1)
    estimator.fit(X,classes)
    conf = estimator.decision_function(X)
    
    train_ids = []
    test_ids = []
    for i,(class_info, conf_info) in enumerate(zip(classes, conf)):
        if abs(conf_info) > confidence_threshold:
            if class_info == 0:
                train_ids.append(i)
            else:
                test_ids.append(i)
    return train_ids, test_ids

Retrieve seed sequences from RFam families.

Use EDeN to transform sequences to vectors. Use RNAfold to create structures and EDeN to convert those to vectors.

Use the sequence vectors to guide the train/test split. 

In [74]:
from sklearn.random_projection import SparseRandomProjection
import time

def rfam_uri(family_id):
        return 'http://rfam.xfam.org/family/%s/alignment?acc=%s&format=fastau&download=0'%(family_id,family_id)

#RNAfold
def pre_processor(seqs):
    from eden.converter.rna.rnafold import rnafold_to_eden
    graphs = rnafold_to_eden(seqs)
    from eden.modifier.graph import structure 
    graphs = structure.basepair_to_nesting(graphs)
    return graphs

    
#RNAVectorizer
def pre_processor(seqs):
    n_neighbors = min(len(seqs),30)
    rs = int(time.time())
    from eden.RNA import Vectorizer as RNAVectorizer
    rnavec=RNAVectorizer(n_neighbors=n_neighbors,
                         sampling_prob=.15,
                         n_iter=5,
                         min_energy=-5,
                         random_state=rs)
    rnavec.fit(seqs)
    graphs = rnavec.graphs(seqs)
    from eden.modifier.graph import structure 
    graphs = structure.basepair_to_nesting(graphs)
    return graphs


def rfam_to_matrix(rfam_id, use_structure=True, n_max=50, complexity=2, nbits=10):
    from eden.converter.fasta import fasta_to_sequence
    seqs = fasta_to_sequence(rfam_uri(rfam_id))
    from itertools import islice
    seqs = islice(seqs,n_max)
    seqs = list(seqs)
    if use_structure:
        graphs = pre_processor(seqs)
        from eden.graph import Vectorizer as GraphVectorizer
        vectorizer = GraphVectorizer(r=3,d=0,
                                     normalization=False,
                                     inner_normalization=False,
                                     nbits=nbits)
        X = vectorizer.transform(graphs)
    else:
        from eden.path import Vectorizer as SeqVectorizer
        vectorizer = SeqVectorizer(r=3,d=20,
                                   normalization=False,
                                   inner_normalization=False,
                                   nbits=nbits)
        X = vectorizer.transform(seqs)
    return X

def rfam_data(rfam_ids, n_max=300, complexity=3, nbits=13):
    import numpy as np
    from scipy.sparse import vstack
    seq_train_list = []
    seq_test_list = []
    struct_train_list = []
    struct_test_list = []
    y_train = []
    y_test = []
    for i,rfam_id in enumerate(rfam_ids):
        # seq case
        seq_X=rfam_to_matrix(rfam_id, use_structure=False, n_max=n_max, complexity=complexity, nbits=nbits)
        train_ids, test_ids = train_test_ids_split(seq_X)
        seq_X_train = seq_X[train_ids]
        seq_X_test = seq_X[test_ids]
        seq_train_list.append(seq_X_train)
        seq_test_list.append(seq_X_test)
        y_train += [i] * seq_X_train.shape[0]
        y_test += [i] * seq_X_test.shape[0]
        
        # struct case
        struct_X=rfam_to_matrix(rfam_id, use_structure=True, n_max=n_max, complexity=complexity, nbits=nbits)
        #NOTE: use the same split given by the sequence similarity
        struct_X_train = struct_X[train_ids]
        struct_X_test = struct_X[test_ids]
        struct_train_list.append(struct_X_train)
        struct_test_list.append(struct_X_test)
    seq_X_train = vstack(seq_train_list, format="csr")
    seq_X_test = vstack(seq_test_list, format="csr")
    struct_X_train = vstack(struct_train_list, format="csr")
    struct_X_test = vstack(struct_test_list, format="csr")
    target_train = np.array(y_train)
    target_test = np.array(y_test)
    
    return seq_X_train,\
        seq_X_test,\
        struct_X_train,\
        struct_X_test,\
        target_train,\
        target_test

rfam_ids=['RF00004','RF00005','RF00015','RF00020','RF00026','RF00169',
          'RF00380','RF00386','RF01051','RF01055','RF01234','RF01699',
          'RF01701','RF01705','RF01731','RF01734','RF01745','RF01750',
          'RF01942','RF01998','RF02005','RF02012','RF02034']

print 'num families:', len(rfam_ids)

num families: 23


In [75]:
data_ids=rfam_ids[0:5]
print 'Experiment using %d families' % len(data_ids)
prefix='f3_c4nb12_'

Experiment using 5 families


In [76]:
%%time
seq_X_train,\
seq_X_test,\
struct_X_train,\
struct_X_test,\
y_train,\
y_test = rfam_data(data_ids, n_max=300, complexity=4, nbits=12)

CPU times: user 1min 44s, sys: 40.1 s, total: 2min 24s
Wall time: 7min 9s


In [77]:
%%time
save_data(seq_X_train, fname=prefix+'seq_X_train.dat')
save_data(seq_X_test, fname=prefix+'seq_X_test.dat')
save_data(struct_X_train, fname=prefix+'struct_X_train.dat')
save_data(struct_X_test, fname=prefix+'struct_X_test.dat')
save_data(y_train, fname=prefix+'y_train.dat')
save_data(y_test, fname=prefix+'y_test.dat')

Saved f3_c4nb12_seq_X_train.dat (575,4097)
Saved f3_c4nb12_seq_X_test.dat (471,4097)
Saved f3_c4nb12_struct_X_train.dat (575,4097)
Saved f3_c4nb12_struct_X_test.dat (471,4097)
Saved f3_c4nb12_y_train.dat (575)
Saved f3_c4nb12_y_test.dat (471)
CPU times: user 14.4 ms, sys: 76.3 ms, total: 90.8 ms
Wall time: 568 ms


In [78]:
%%time
seq_X_train=load_data(fname=prefix+'seq_X_train.dat')
seq_X_test=load_data(fname=prefix+'seq_X_test.dat')
struct_X_train=load_data(fname=prefix+'struct_X_train.dat')
struct_X_test=load_data(fname=prefix+'struct_X_test.dat')
y_train=load_data(fname=prefix+'y_train.dat')
y_test=load_data(fname=prefix+'y_test.dat')

Loaded f3_c4nb12_seq_X_train.dat (575,4097)
Loaded f3_c4nb12_seq_X_test.dat (471,4097)
Loaded f3_c4nb12_struct_X_train.dat (575,4097)
Loaded f3_c4nb12_struct_X_test.dat (471,4097)
Loaded f3_c4nb12_y_train.dat (575)
Loaded f3_c4nb12_y_test.dat (471)
CPU times: user 15 ms, sys: 38.5 ms, total: 53.5 ms
Wall time: 52.6 ms


In [79]:
%%time
# sparse to dense arrays
Xseq_train = seq_X_train
Xseq_test = seq_X_test
X_train = struct_X_train
X_test = struct_X_test

CPU times: user 50 µs, sys: 5.15 ms, total: 5.2 ms
Wall time: 5.19 ms


In [80]:
%%time
# sparse to dense arrays
Xseq_train = seq_X_train.toarray()
Xseq_test = seq_X_test.toarray()
X_train = struct_X_train.toarray()
X_test = struct_X_test.toarray()

CPU times: user 34.7 ms, sys: 20.7 ms, total: 55.5 ms
Wall time: 54.9 ms


#Sequence features

In [81]:
%%time
train_perform(Xseq_train, y_train, Xseq_test,y_test)

Training on data matrix [575 x 4097]
Confusion matrix:
[[ 43   6   3   2   0]
 [  0  32   1  13  18]
 [ 15  62  24   4   0]
 [  0  27   8  46   1]
 [  2  27   5   1 131]]

Classification Report:
             precision    recall  f1-score   support

          0       0.72      0.80      0.75        54
          1       0.21      0.50      0.29        64
          2       0.59      0.23      0.33       105
          3       0.70      0.56      0.62        82
          4       0.87      0.79      0.83       166

avg / total       0.67      0.59      0.60       471

CPU times: user 186 ms, sys: 6.57 ms, total: 192 ms
Wall time: 194 ms


#Structure features

In [82]:
%%time
train_perform(X_train, y_train, X_test,y_test)

Training on data matrix [575 x 4097]
Confusion matrix:
[[ 54   0   0   0   0]
 [  7  23   6   4  24]
 [ 15  31  59   0   0]
 [ 10  22   3  46   1]
 [  4 102   3   1  56]]

Classification Report:
             precision    recall  f1-score   support

          0       0.60      1.00      0.75        54
          1       0.13      0.36      0.19        64
          2       0.83      0.56      0.67       105
          3       0.90      0.56      0.69        82
          4       0.69      0.34      0.45       166

avg / total       0.67      0.51      0.54       471

CPU times: user 205 ms, sys: 4.76 ms, total: 210 ms
Wall time: 134 ms


#Learned map seq $\mapsto$ structure features

In [83]:
from sklearn.preprocessing import StandardScaler
seqs_scale = StandardScaler(with_mean=True)
data_matrix_in = seqs_scale.fit_transform(Xseq_train)
struct_scale = StandardScaler(with_mean=True)
data_matrix_out = struct_scale.fit_transform(X_train)
n_features_in = data_matrix_in.shape[1]
n_features_out = data_matrix_out.shape[1]
n_features_hidden = max(n_features_in, n_features_out) / 2
print 'n_neurons: #in [%d] -- #hidden [%d] -- #out [%d]' % (n_features_in, n_features_hidden, n_features_out)

n_neurons: #in [4097] -- #hidden [2048] -- #out [4097]


In [84]:
from sknn.mlp import Regressor, Layer

net = Regressor(layers=[
        Layer("Rectifier", units=n_features_hidden),
        Layer("Rectifier", units=n_features_hidden),
        Layer("Rectifier", units=n_features_hidden),
        Layer("Linear", units=n_features_out)],
                learning_rate=0.0001,
                n_iter=40,
                batch_size=10,
                regularize='L1',
                valid_size=0.1)

In [85]:
%%time
net.fit(data_matrix_in, data_matrix_out)

CPU times: user 20min 38s, sys: 1min 32s, total: 22min 10s
Wall time: 10min 42s


Regressor(batch_size=10, debug=False, dropout_rate=None, f_stable=0.001,
     hidden0=<sknn.nn.Layer `Rectifier`: name=u'hidden0', units=2048>,
     hidden1=<sknn.nn.Layer `Rectifier`: name=u'hidden1', units=2048>,
     hidden2=<sknn.nn.Layer `Rectifier`: name=u'hidden2', units=2048>,
     layers=[<sknn.nn.Layer `Rectifier`: name=u'hidden0', units=2048>, <sknn.nn.Layer `Rectifier`: name=u'hidden1', units=2048>, <sknn.nn.Layer `Rectifier`: name=u'hidden2', units=2048>, <sknn.nn.Layer `Linear`: name=u'output', units=4097>],
     learning_momentum=0.9, learning_rate=0.0001, learning_rule=u'sgd',
     loss_type=u'mse', mutator=None, n_iter=40, n_stable=50,
     output=<sknn.nn.Layer `Linear`: name=u'output', units=4097>,
     random_state=None, regularize='L1',
     valid_set=(array([[ 0.     , -1.21392, ...,  0.62572,  0.43704],
       [ 0.     ,  0.61331, ...,  0.62572, -1.23827],
       ...,
       [ 0.     ,  0.61331, ...,  0.62572,  0.43704],
       [ 0.     ,  0.61331, ..., -1.34033,

In [86]:
%%time
# Transform seq features to struct features
X_train_pred = net.predict(seqs_scale.transform(Xseq_train))
X_test_pred = net.predict(seqs_scale.transform(Xseq_test))

CPU times: user 4.78 s, sys: 87.4 ms, total: 4.87 s
Wall time: 804 ms


In [87]:
%%time
train_perform(X_train_pred, y_train, X_test_pred, y_test)

Training on data matrix [575 x 4097]
Confusion matrix:
[[ 54   0   0   0   0]
 [  0  63   0   1   0]
 [ 46  11  36  11   1]
 [ 14  15   8  45   0]
 [  6 120  13  11  16]]

Classification Report:
             precision    recall  f1-score   support

          0       0.45      1.00      0.62        54
          1       0.30      0.98      0.46        64
          2       0.63      0.34      0.44       105
          3       0.66      0.55      0.60        82
          4       0.94      0.10      0.17       166

avg / total       0.68      0.45      0.40       471

CPU times: user 181 ms, sys: 8.63 ms, total: 189 ms
Wall time: 136 ms


---